In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

In [4]:
train = pd.read_csv('train.csv',parse_dates=['fecha']).drop(columns=['lat','lng'])
test = pd.read_csv('test.csv',parse_dates=['fecha']).drop(columns=['lat','lng'])

In [5]:
pd.set_option('display.max_colwidth',-1)
train.descripcion

0         depto. interior de 80.15m2, consta de sala comedor cocina equipada, área de lavado, baño completo, recamara principal con closet y baño completo. 2da. recamara con closet.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [3]:
#Arreglo problema de metros totales y metros cubiertos
train['metrostotales_reales'] = train['metrostotales'].where(train.metrostotales>=train.metroscubiertos,train.metroscubiertos)
train.metroscubiertos = train['metroscubiertos'].where(train.metrostotales>=train.metroscubiertos,train.metrostotales)
train.metrostotales = train.metrostotales_reales
train.drop(columns=['metrostotales_reales'],inplace=True)

In [4]:
#Arreglo problema de metros totales y metros cubiertos
test['metrostotales_reales'] = test['metrostotales'].where(test.metrostotales>=test.metroscubiertos,test.metroscubiertos)
test.metroscubiertos = test['metroscubiertos'].where(test.metrostotales>=test.metroscubiertos,test.metrostotales)
test.metrostotales = test.metrostotales_reales
test.drop(columns=['metrostotales_reales'],inplace=True)

In [5]:
train.tipodepropiedad.isnull().sum()

46

In [6]:
cat_features = ['tipodepropiedad', 'provincia', 'ciudad']
num_features = ['banos','garages','habitaciones','metrostotales','metroscubiertos','antiguedad']
bool_features = ['gimnasio','usosmultiples','piscina','escuelascercanas','centroscomercialescercanos']

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [8]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
transformador_numeros = SimpleImputer()

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, num_features),
        ('cat', transformador_categorias, cat_features)
    ])

In [10]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=2000, learning_rate=0.05, max_depth=10, subsample_by_tree=0.5, colsample_by_col=0.5, min_child_weight=3)
from catboost import CatBoostRegressor
model2 = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.05, loss_function='RMSE', task_type="GPU")

In [11]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])
pipeline2 = Pipeline(steps =[('preprocessor', preprocessor),
                             ('model', model2)])

In [12]:
features = ['tipodepropiedad', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'provincia','ciudad'] 
train_y = train.precio.copy()
train_x = train[features].copy()

In [ ]:
my_pipeline.fit(train_x, train_y)

In [ ]:
preds = my_pipeline.predict(test_x)

In [ ]:
res = pd.DataFrame(preds, index=test.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)